In [1]:
import os
import requests
import pandas as pd
import redshift_connector
from dotenv import dotenv_values

# Configurar pandas para mostrar todas las filas y columnas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Configurar variables de entorno
env_vars = dotenv_values('.env')

# Test PokeAPI
response = requests.get("https://pokeapi.co/api/v2/pokemon/1")
print(response)

<Response [200]>


In [2]:
data = response.json()
data.keys()

dict_keys(['abilities', 'base_experience', 'forms', 'game_indices', 'height', 'held_items', 'id', 'is_default', 'location_area_encounters', 'moves', 'name', 'order', 'past_types', 'species', 'sprites', 'stats', 'types', 'weight'])

In [3]:
pokemon_df = pd.DataFrame()

for i in range(151):
    response = requests.get(f"https://pokeapi.co/api/v2/pokemon/{i+1}")
    data = response.json()

    row_df = pd.DataFrame()  # (index=[0])

    row_df.loc[0,'id'] = data['id'] 
    row_df.loc[0,'name'] = data['name']
    row_df.loc[0,'height'] = data['height']
    row_df.loc[0,'weight'] = data['weight']
    row_df.loc[0,'main_type'] = data['types'][0]['type']['name']
    row_df.loc[0,'hp'] = data['stats'][0]['base_stat']
    row_df.loc[0,'attack'] = data['stats'][1]['base_stat']
    row_df.loc[0,'defense'] = data['stats'][2]['base_stat']
    row_df.loc[0,'special_attack'] = data['stats'][3]['base_stat']
    row_df.loc[0,'special_defense'] = data['stats'][4]['base_stat']
    row_df.loc[0,'speed'] = data['stats'][5]['base_stat']
            
    
    pokemon_df = pd.concat([pokemon_df, row_df], ignore_index=True)

pokemon_df = pokemon_df.astype({'id':'int', 'name':'str', 'height':'int', 'weight':'int', 'main_type':'str', 'hp':'int', 
                                'attack':'int', 'defense':'int', 'special_attack':'int', 'special_defense':'int', 'speed':'int'}, errors='raise')

pokemon_df.head()


,id,name,height,weight,main_type,hp,attack,defense,special_attack,special_defense,speed
0,1,bulbasaur,7,69,grass,45,49,49,65,65,45
1,2,ivysaur,10,130,grass,60,62,63,80,80,60
2,3,venusaur,20,1000,grass,80,82,83,100,100,80
3,4,charmander,6,85,fire,39,52,43,60,50,65
4,5,charmeleon,11,190,fire,58,64,58,80,65,80


In [5]:
env_vars['HOST']

'data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com'

In [7]:
conn = redshift_connector.connect(
    host=env_vars['HOST'],
    port=int(env_vars['PORT']),
    database=env_vars['DATABASE'],
    user=env_vars['USER'],
    password=env_vars['PASSWORD']
 )

# Create a Cursor object
cursor = conn.cursor()


In [ ]:
# Eliminate the table if exists
cleaner_query = '''DROP TABLE IF EXISTS laureanoengulian_coderhouse.pokemon_first_gen'''
cursor.execute(cleaner_query)
conn.commit()

# Create table if not exists
create_table_query = '''CREATE TABLE IF NOT EXISTS laureanoengulian_coderhouse.pokemon_first_gen(
                        id integer not null,
                        name varchar(150) not null,
                        height integer not null,
                        weight integer not null,
                        main_type varchar(150) not null,
                        hp integer not null,
                        attack integer not null,
                        defense integer not null,
                        special_attack integer not null,
                        special_defense integer not null,
                        speed integer not null,
                        primary key(id))
                     '''
cursor.execute(create_table_query)
conn.commit()

# Load the df in Redshift
cursor.write_dataframe(df=pokemon_df, table='pokemon_first_gen')
conn.commit()